# Задание Pro к занятию Методы Faiss

Возьмите несколько (от двух и более) любых текстовых документов, на основании информации из которых вы создадите консультанта.

Можете взять две части баз знаний компании Simble (без разбивки MarkDown). Ссылки даны ниже.

url_base1_simble = '[https://docs.google.com/document/d/1Z7eZLIPG9URgOFz-yqtJAup-WXhKFIiF](https://docs.google.com/document/d/1Z7eZLIPG9URgOFz-yqtJAup-WXhKFIiF)'

url_base2_simble = '[https://docs.google.com/document/d/1qxJXwHtYNxx6ecf35zhqFYBxSjoA5Mhr](https://docs.google.com/document/d/1qxJXwHtYNxx6ecf35zhqFYBxSjoA5Mhr)'


- Разбейте каждый текст на чанки любым удобным и подходящим способом
- Создайте отдельные векторные базы на каждую часть и сохраниете их на ГуглДиск
- Загрузите из ГуглДиска сохраненные на предыдущем шаге отдельные векторные базы и объедините их в одну.
- Создайте простого консультанта по объединенной базе знаний. Используйте ретривер и ChatOpenAI из Langchain.
- Задайте вопрос консультанту по базе знаний и получите ответ

In [1]:
# Устанавливаем необходимые библиотеки
!pip install -q openai==1.57.0 langchain==0.3.9 langchain_core==0.3.21 langchain_community==0.3.9 \
langchain-openai==0.2.11 faiss-cpu==1.9.0.post1 tiktoken==0.8.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.9/389.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00


In [2]:
# Импортируем необходимые библиотеки
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
import os
import re
import requests
import textwrap
import openai
import tiktoken
import matplotlib.pyplot as plt
from openai import OpenAI
from google.colab import userdata, drive
from langchain.indexes import VectorstoreIndexCreator
from langchain.chains import RetrievalQA
from langchain_openai import OpenAI
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

In [3]:
# Использование ключа API от ProxyAPI
key = userdata.get('PROXY_API_KEY')
os.environ["OPENAI_API_KEY"] = key

# Адрес сервера ProxyAPI
base_url = 'https://api.proxyapi.ru/openai/v1'
os.environ["OPENAI_BASE_URL"] = base_url

# Создание клиента OpenAI с использованием API ключа из переменных среды
client = OpenAI()

In [4]:
# Функции

# функция для загрузки документа Docx по ссылке из гугл драйв
def download_google_doc(url: str) -> str:
    # Извлекаем ID документа из URL
    match_ = re.search(r'/document/d/([a-zA-Z0-9-_]+)', url)
    if not match_:
        raise ValueError("Invalid Google Docs URL")
    doc_id = match_.group(1)
    download_url = f'https://docs.google.com/document/d/{doc_id}/export?format=txt'
    response = requests.get(download_url, stream=True)
    if response.status_code != 200:
        raise RuntimeError("Failed to download the document")
    return response.text


# Функция для форматирования текста по абзацам
def format_text(text, width=120):
    # Разделяем текст на абзацы
    paragraphs = text.split('\n')
    # Форматируем каждый абзац отдельно
    formatted_paragraphs = []
    for paragraph in paragraphs:
        # Используем textwrap.fill для форматирования абзаца, чтобы длина строки не превышала width
        formatted_paragraph = textwrap.fill(paragraph, width)
        formatted_paragraphs.append(formatted_paragraph)
    # Объединяем абзацы с символом новой строки
    return '\n'.join(formatted_paragraphs)


# Функция возвращает количество токенов в строке в зависимости от используемой модели
def num_tokens_from_string(string: str, model='gpt-4o-mini') -> int:
    # Получаем имя кодировки для указанной модели
    encoding_name = tiktoken.encoding_for_model(model).name
    # Получаем объект кодировки на основе имени кодировки
    encoding = tiktoken.get_encoding(encoding_name)
    # Кодируем строку и вычисляем количество токенов
    num_tokens = len(encoding.encode(string))
    # Возвращаем количество токенов
    return num_tokens


# Построение гистограммы распределения количества токенов по чанкам
def create_histogram(chunks):
    print("\nОбщее количество чанков: ", len(chunks))
    # Подсчет токенов для каждого чанка
    try: # для формата чанков LangChain Document
        chunk_token_counts = [num_tokens_from_string(chunk.page_content) for chunk in chunks]
    except: # для текстового формата чанков
        chunk_token_counts = [num_tokens_from_string(chunk) for chunk in chunks]
    # Строим гистограмму
    plt.figure(figsize=(7, 4)) # размер
    plt.hist(chunk_token_counts, bins=10, alpha=0.5, label='Чанки')
    plt.title('Распределение к-ва токенов по чанкам')  # Заголовок графика
    plt.xlabel('К-во токенов в чанке')  # Подпись оси X
    plt.ylabel('К-во чанков')  # Подпись оси Y
    plt.show()  # Отображаем график


# (CharacterTextSplitter) Формируем чанки из текста по количеству символов
def split_text(text: str,
               chunk_size=3000,    # Ограничение к-ва символов в чанке
               chunk_overlap=200): # к-во символов перекрытия в чанке
    # Удалить пустые строки и лишние пробелы
    text = re.sub(r'\s+', ' ', text).strip()
    # Создаем экземпляр CharacterTextSplitter с заданными парамаетрами
    splitter = CharacterTextSplitter(chunk_size=chunk_size,
                                     chunk_overlap=chunk_overlap,
                                     separator=" ")
    return splitter.split_text(text) # список текстовых чанков


# Создание индексной (векторной) базы из текстовых чанков и сохранение на диск
def create_db_index_from_text_save(chunks_text,
                                   index_name, # имя для индексной базы
                                   path):  # путь к папке
    # Создаем индексную базу с использованием FAISS ---------------------------------
    # FAISS.from_texts - из списка текстовых чанков
    db_index = FAISS.from_texts(chunks_text, OpenAIEmbeddings())
    # сохраняем индексную базу ------------------------------------------------------
    db_index.save_local(folder_path=path, # путь к папке (path)
                        index_name=index_name)  # имя для индексной базы (index_name)
    # --------------------------------------------------------------------------------
    return db_index


# Загрузка векторной базы с диска
def load_db_vector(folder_path_db_index,  # путь к сохраненной векторной базе
                   index_name):           # имя сохраненной векторной базы
    return FAISS.load_local(
                allow_dangerous_deserialization=True,
                embeddings=OpenAIEmbeddings(),
                folder_path=folder_path_db_index,
                index_name=index_name)


# Функция запроса и ответа от OpenAI с поиском по векторной базе данных
def generate_answer(query,    # запрос пользователя
                    db_index, # векторная база знаний
                    k=5,      # используемое к-во чанков
                    verbose=True, # выводить ли на экран выбранные чанки
                    model='gpt-4o-mini',
                    temp=0.1):
    # Поиск чанков по векторной базе данных
    similar_documents = db_index.similarity_search(query, k=k)
    # Формирование текстового контента из выбранных чанков для модели
    message_content = re.sub(r'\n{2}', ' ', '\n '.join(
        [f'Отрывок документа № {i+1}:\n' + doc.page_content
        for i, doc in enumerate(similar_documents)]))
    if verbose:
        print(message_content) # печать на экран выбранных чанков

    messages = [{"role": "system",
                 "content": f'Ответь на вопрос пользователя на основании информации из базы знаний: \n{message_content}'},
                {"role": "user",
                 "content": f'Вопрос пользователя: {query}'}]
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temp)
    return response.choices[0].message.content

In [5]:
data_from_url_1 = download_google_doc('https://docs.google.com/document/d/1Z7eZLIPG9URgOFz-yqtJAup-WXhKFIiF')

In [6]:
formatted_data = format_text(data_from_url_1)
source_chank = split_text(formatted_data)

db_index_1 = create_db_index_from_text_save(source_chank, 'db_index_1', '/content')

In [7]:
data_from_url_2 = download_google_doc('https://docs.google.com/document/d/1qxJXwHtYNxx6ecf35zhqFYBxSjoA5Mhr')

In [8]:
formatted_data = format_text(data_from_url_2)
source_chank = split_text(formatted_data)

db_index_2 = create_db_index_from_text_save(source_chank, 'db_index_2', '/content')

In [9]:
load_db_index_1 = load_db_vector('/content', 'db_index_1')
load_db_index_2 = load_db_vector('/content', 'db_index_2')

In [10]:
load_db_index_1.merge_from(load_db_index_2)

In [11]:
# можно указать параметры, в данном случае, укажем search_type="mmr"
qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(model_name="gpt-4o-mini", temperature=0, max_tokens=400), retriever=load_db_index_1.as_retriever(search_type="mmr"))

In [12]:
query = "Как оформить страховку в Simble?"
qa.run(query)

<ipython-input-12-b4a83741de45>:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  qa.run(query)


'Чтобы оформить страховку в Simble, необходимо выполнить следующие шаги:\n\n1. Скачайте приложение Simble на устройство с операционной системой iOS или Android.\n2. Зарегистрируйтесь в приложении и укажите свои данные, а также данные о вашем автомобиле. Если у вас уже есть автомобиль, вы можете найти его по номеру или указать данные ПТС. Если автомобиля нет, нужно пройти фотоосмотр и сфотографировать машину со всех сторон. Это нужно сделать только один раз, при первой покупке.\n3. После прохождения фотоосмотра ваша страховка будет одобрена в течение примерно 3 часов.\n4. Затем выберите тариф, который наиболее подходит вам, и купите полис.\n\nЕсли у вас есть дополнительные вопросы, вы можете обратиться в службу поддержки Simble.'

In [13]:
query = "Что такое Simble?"
qa.run(query)

'Simble - это IT-продукт, который предлагает услуги по управлению автостраховкой. Он позволяет пользователям оформлять страховые полисы, такие как КАСКО и ОСАГО, на короткий срок, а также предоставляет возможность выбора тарифов и сравнения цен от различных страховых компаний. Приложение доступно для установки на устройства с операционными системами iOS и Android. Simble также сотрудничает с различными партнерами и предлагает программы лояльности для своих пользователей.'